In [1]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


from keys import Trip_advisor_key
from keys import geoapify_key

In [4]:

# Defining the District dataframe

Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto South West', 'Toronto North West', 'Toronto Yorkville', 'Ottawa City Centre' , 'Niagara Falls'],
    'Lat': [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236, 43.087586], 
    'Lng': [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009, -79.081206]  
})


# Defining the Restaurant DataFrame

Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Postal Code', 'Restaurant Address', 'Lat', 'Lng'])

# Loop for districts
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    for restaurant in response.get('features', []):
        try:
            
            new_row = {
                            'Location': row['Location'],
                            'Restaurant Name': restaurant["properties"]["address_line1"],
                            'Postal Code' : restaurant["properties"]["postcode"],
                            'Restaurant Address': restaurant["properties"]["address_line2"],
                            'Lat': restaurant["geometry"]["coordinates"][1],
                            'Lng': restaurant["geometry"]["coordinates"][0]
                        }
            
            Restaurants_df.loc[len(Restaurants_df)] = new_row
        except(KeyError, IndexError):
            print('Errore nella gestione dei dati del ristorante.')


In [8]:
Restaurants_df.head()

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538


In [6]:
# Export the City_Data into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [35]:
Restaurants_df['Location ID'] = ''

err_count = 0

for index, restaurant in Restaurants_df.iterrows():
    
    url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery={restaurant['Restaurant Name']}&language=en"

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers).json()
    
    try:
        
        for res in response['data']:
            
            #pprint(res['address_obj']['postalcode'])
    
            if res['address_obj']['postalcode'] == restaurant['Postal Code']:
            
                Restaurants_df.at[index, 'Location ID'] = res['location_id']
                
                break
    
    except Exception as e:
        
        err_count = err_count +1
        
        
            
    
        

#print(response.text)
print(err_count)
Restaurants_df.head()

49


,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811,
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538,


In [43]:
Restaurants_df.head(10)

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811,
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538,
5,Toronto Distillery District,Cluny,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650604,-79.357865,6762792
6,Toronto Distillery District,Mom and Pops,M5A 4J5,"262 The Esplanade, Old Toronto, ON M5A 4J5, Ca...",43.650014,-79.363850,
7,Toronto Distillery District,Gilead Café + Wine Bar,M5A 3C9,"4 Gilead Place, Old Toronto, ON M5A 3C9, Canada",43.653558,-79.360945,
8,Toronto Distillery District,Freshii,M5A 1L1,"351 King Street East, Old Toronto, ON M5A 1L1,...",43.651300,-79.364434,
9,Toronto Distillery District,The Corktown Kitchen,M5A 1L3,"King Street East, Old Toronto, ON M5A 1L3, Canada",43.653422,-79.362286,


In [52]:
Restaurants_df = Restaurants_df[Restaurants_df['Location ID'] != '']
Restaurants_df

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
5,Toronto Distillery District,Cluny,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650604,-79.357865,6762792
25,Toronto South West,Azure Restaurant & Bar,M5V 2X3,"225 Front Street West, Old Toronto, ON M5V 2X3...",43.644434,-79.385390,682125
26,Toronto South West,SOCO Kitchen + Bar,M5J 3A6,"75 Lower Simcoe Street, Old Toronto, ON M5J 3A...",43.642679,-79.383845,7243578
33,Toronto South West,Buster's Sea Cove,M5J 0A8,"PATH, Old Toronto, ON M5J 0A8, Canada",43.643164,-79.382862,19815927
43,Toronto North West,Café Diplomatico,M6G 1B3,"594 College Street, Old Toronto, ON M6G 1B3, C...",43.655369,-79.413731,683135
47,Toronto North West,Hapa Izakaya,M6G 1B4,"602 College Street, Old Toronto, ON M6G 1B4, C...",43.655255,-79.414264,3506716
57,Toronto North West,Duff's Famous Wings,M6G 1B1,"558 College Street, Old Toronto, ON M6G 1B1, C...",43.655711,-79.412207,1510697


In [53]:
# Export the City_Data into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [46]:
url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery=Mom and Pops&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
    
print(response.text)

{
  "data": [
    {
      "location_id": "4887334",
      "name": "Mom & Pop's",
      "address_obj": {
        "street1": "629 N Riverfront Dr",
        "street2": "Corner of Rock St. & N. Riverfront Dr. -- Near the entrance to Riverfront Park",
        "city": "Mankato",
        "state": "Minnesota",
        "country": "United States",
        "postalcode": "56001-3452",
        "address_string": "629 N Riverfront Dr Corner of Rock St. & N. Riverfront Dr. -- Near the entrance to Riverfront Park, Mankato, MN 56001-3452"
      }
    },
    {
      "location_id": "3387185",
      "name": "Mom and Pop's Diner",
      "address_obj": {
        "street1": "224 S Carson St",
        "city": "Carson City",
        "state": "Nevada",
        "country": "United States",
        "postalcode": "89701-4743",
        "address_string": "224 S Carson St, Carson City, NV 89701-4743"
      }
    },
    {
      "location_id": "12161289",
      "name": "Mom & Pops Burgers",
      "address_obj": {
       

In [2]:
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

latitude = 43.6503
longitude = -79.3596

params['filter'] = f'circle:{longitude},{latitude},{radius}'
params['bias'] = f'proximity:{longitude},{latitude}'

response = requests.get(base_url, params=params).json()

pprint(response)

{'features': [{'geometry': {'coordinates': [-79.3596299, 43.65068200054068],
                            'type': 'Point'},
               'properties': {'address_line1': 'Archeo Trattoria',
                              'address_line2': '31 Trinity Street, Old '
                                               'Toronto, ON M5A 3C4, Canada',
                              'categories': ['catering',
                                             'catering.restaurant',
                                             'wheelchair',
                                             'wheelchair.limited'],
                              'city': 'Old Toronto',
                              'country': 'Canada',
                              'country_code': 'ca',
                              'county': 'Golden Horseshoe',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                    